# Inference Service

* Create an inference service to serve one of the registered models
* Fetch online features and calculate predictions from the inference service

In [1]:
!pip install --upgrade pip
!cat requirements.`txt
!pip install -q -r requirements.txt
!pip install --no-deps --ignore-requires-python "https://github.com/opendatahub-io/ml-metadata/releases/download/v1.14.0%2Bremote.1/ml_metadata-1.14.0+remote.1-py3-none-any.whl" # need a Python 3.11 compatible version
!pip install --no-deps --ignore-requires-python "model-registry==0.1.2" # ignore dependencies because of the above override

feast==0.36.0
psycopg2>=2.9
onnx==1.14.1
onnxruntime
#tf2onnx
#typing-extensions==4.5.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.7 MB/s eta 0:00:00


## Imports and constants

In [9]:
import os

import boto3
import matplotlib.pyplot as plt
import onnx
import onnxruntime as ort
from feast import FeatureStore
from model_registry import ModelRegistry
from model_registry.types import ContextState

In [10]:
def modelRegistry():
    return ModelRegistry(server_address="modelregistry-sample.feast.svc.cluster.local", port=9090, author="feast-dev@redhat.com")

In [12]:
registry = modelRegistry()

models = registry._api.get_registered_models()
for model in models:
    print(f"Model {model.name}")
    model_versions = registry._api.get_model_versions(model.id)
    for model_version in model_versions:
        print(f"Version {model_version.name}")
        
# Update the logic to select the model and the version
selected_model = models[0]
live_model_versions = [m for m in registry._api.get_model_versions(selected_model.id) if m.state!=ContextState.ARCHIVED]
selected_model_version = live_model_versions[0]
selected_model_artifact = registry._api.get_model_artifacts(selected_model_version.id)[0]

print(f"Creating inference service for model: {selected_model}")
print(f"Model version: {selected_model_version}")
print(f"Model artifact: {selected_model_artifact}")

Model mnist
Version v.simple_NN.20240417155921
Version v.convolutedNN.202404171600521
Version v.simple_NN.20240418124507
Version v.simple_NN.20240419072531
Version v.convolutedNN.202404190726401
Version v.simple_NN.20240419073913
Version v.convolutedNN.202404190740471
Creating inference service for model: RegisteredModel(id='1', description='', external_id='', create_time_since_epoch=1713369565611, last_update_time_since_epoch=1713369565611, state=<ContextState.LIVE: 'LIVE'>, name='mnist')
Model version: ModelVersion(name='v.simple_NN.20240419073913', id='7', description='demo20231121 e2e MNIST', external_id='', create_time_since_epoch=1713512361312, last_update_time_since_epoch=1713512361312, state=<ContextState.LIVE: 'LIVE'>, model_name='mnist', version='v.simple_NN.20240419073913', author='feast-dev@redhat.com', metadata={}, artifacts=NOTHING, _registered_model_id=NOTHING)
Model artifact: ModelArtifact(id='6', description='', external_id='', create_time_since_epoch=1713512362011, la